### Predictive Model with SVM

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

#Load libraries for data processing
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from scipy.stats import norm

## Supervised learning.
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn import metrics, preprocessing
from sklearn.metrics import classification_report

from scipy.io import loadmat 

# visualization
import seaborn as sns 
plt.style.use('fivethirtyeight')
sns.set_style("white")

plt.rcParams['figure.figsize'] = (8,4)

In [2]:
xx=loadmat('./mu_30_new.mat')

In [3]:
xx

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sun Oct 29 19:07:36 2023',
 '__version__': '1.0',
 '__globals__': [],
 'mu_30': array([[-1.38971000e-01,  8.07940000e-02,  4.67550000e-02, ...,
          3.08949880e+01,  7.56606660e+01,  2.00000000e+00],
        [-1.14305000e-01,  2.10289000e-01,  1.64010000e-02, ...,
          3.08949880e+01,  7.56421660e+01,  2.00000000e+00],
        [ 1.87850000e-01,  2.45232000e-01, -3.83500000e-03, ...,
          3.08949880e+01,  7.56668320e+01,  2.00000000e+00],
        ...,
        [-3.82520000e-02,  4.79070000e-02,  1.25651595e+02, ...,
          0.00000000e+00,  7.54818400e+01,  2.00000000e+00],
        [-7.52510000e-02, -4.66450000e-02,  1.25945017e+02, ...,
          0.00000000e+00,  7.52290160e+01,  2.00000000e+00],
        [-4.23630000e-02, -7.95320000e-02,  1.25955135e+02, ...,
          0.00000000e+00,  7.52043510e+01,  2.00000000e+00]])}

In [4]:
xx.keys()

dict_keys(['__header__', '__version__', '__globals__', 'mu_30'])

In [5]:
df=xx['mu_30']
df2=xx['mu_30']

In [6]:
X=df[:,:8]

In [7]:
y=df2[:,-1]

In [8]:
X.shape

(4300, 8)

In [9]:
y.shape

(4300,)

In [10]:
X

array([[-1.38971000e-01,  8.07940000e-02,  4.67550000e-02, ...,
         8.10446970e+01,  3.10369250e+01,  3.08949880e+01],
       [-1.14305000e-01,  2.10289000e-01,  1.64010000e-02, ...,
         8.10532790e+01,  3.10842370e+01,  3.08949880e+01],
       [ 1.87850000e-01,  2.45232000e-01, -3.83500000e-03, ...,
         8.10618610e+01,  3.11315500e+01,  3.08949880e+01],
       ...,
       [-3.82520000e-02,  4.79070000e-02,  1.25651595e+02, ...,
         9.33903910e+01,  0.00000000e+00,  0.00000000e+00],
       [-7.52510000e-02, -4.66450000e-02,  1.25945017e+02, ...,
         9.33903910e+01,  0.00000000e+00,  0.00000000e+00],
       [-4.23630000e-02, -7.95320000e-02,  1.25955135e+02, ...,
         9.33903910e+01,  0.00000000e+00,  0.00000000e+00]])

In [11]:
y

array([2., 2., 2., ..., 2., 2., 2.])

In [12]:
import collections
frequency = collections.Counter(y)
print(dict(frequency))

{2.0: 3138, 1.0: 1162}


In [24]:
from scipy import stats
stats.describe(X)

DescribeResult(nobs=4300, minmax=(array([-5.115273, -2.595432, -0.347847, -0.434947, 83.908095, 81.044697,
       -1.324747, -2.649494]), array([  4.403624,   1.93072 , 136.133849, 132.731328,  95.285252,
        93.39181 ,  31.178862,  32.92942 ])), mean=array([-2.29577500e-02, -1.25899693e-02,  7.20929507e+01,  6.31970632e+01,
        9.30161488e+01,  9.09173183e+01,  9.53148713e+00,  1.03431146e+01]), variance=array([2.93862299e-01, 1.04557303e-01, 2.29999727e+03, 2.64418231e+03,
       1.01269701e+01, 1.25568996e+01, 1.43500580e+02, 1.50276617e+02]), skewness=array([-0.42847107, -0.17616163, -0.10231038,  0.18436985, -1.29680758,
       -1.29850505,  0.82581391,  0.65702063]), kurtosis=array([12.04181825,  5.70255427, -1.40829526, -1.640806  ,  0.48330822,
        0.38795558, -0.95821393, -1.24106579]))

In [25]:
#transform the class labels from their original string representation (M and B) into integers
le = LabelEncoder()
y = le.fit_transform(y)

# Normalize the  data (center around 0 and scale to remove the variance).
scaler =StandardScaler()
Xs = scaler.fit_transform(X)

In [26]:
Xs

array([[-0.21403567,  0.28883223, -1.50244263, ..., -2.78638917,
         1.79544442,  1.67670329],
       [-0.16852877,  0.68935417, -1.50307563, ..., -2.78396704,
         1.7993944 ,  1.67670329],
       [ 0.38892436,  0.79743123, -1.50349763, ..., -2.78154491,
         1.80334447,  1.67670329],
       ...,
       [-0.02821673,  0.18711428,  1.11690548, ...,  0.69798513,
        -0.7957641 , -0.84383228],
       [-0.09647709, -0.10533061,  1.12302446, ...,  0.69798513,
        -0.7957641 , -0.84383228],
       [-0.03580122, -0.20704855,  1.12323546, ...,  0.69798513,
        -0.7957641 , -0.84383228]])

In [27]:
# compare the number of repeats for repeated k-fold cross-validation
from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot

In [ ]:
# evaluate a model with a given number of repeats
clf = SVC(probability=True)
def evaluate_model_acc(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores
 
def evaluate_model_apr(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='average_precision', cv=cv, n_jobs=-1)
    return scores

def evaluate_model_f1(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='f1', cv=cv, n_jobs=-1)
    return scores

def evaluate_model_recall(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='recall', cv=cv, n_jobs=-1)
    return scores

# configurations to test
repeats = range(1,4)
results_acc = list()
results_apr = list()
results_f1 = list()
results_recall = list()

for r in repeats:
    # evaluate using a given number of repeats
    scores_acc = evaluate_model_acc(Xs, y, r)
    scores_apr = evaluate_model_apr(Xs, y, r)
    scores_f1 = evaluate_model_f1(Xs, y, r)
    scores_recall = evaluate_model_recall(Xs, y, r)
    # summarize
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_acc), sem(scores_acc), std(scores_acc)))
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_apr), sem(scores_apr), std(scores_apr)))
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_f1), sem(scores_f1), std(scores_f1)))
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_recall), sem(scores_recall),std(scores_recall)))
    # store
    results_acc.append(scores_acc)
    results_apr.append(scores_apr)
    results_f1.append(scores_f1)
    results_recall.append(scores_recall)
# plot the results
pyplot.boxplot(results_acc, labels=[str(r) for r in repeats], showmeans=True)
pyplot.show()

In [28]:
clf = SVC(probability=True)
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3, random_state=2, stratify=y)
y_pred = clf.fit(X_train, y_train).predict(X_test)
cm = metrics.confusion_matrix(y_test, y_pred)
cm

array([[207, 142],
       [ 37, 904]])

In [29]:
print(classification_report(y_test, y_pred ))

              precision    recall  f1-score   support

           0       0.85      0.59      0.70       349
           1       0.86      0.96      0.91       941

    accuracy                           0.86      1290
   macro avg       0.86      0.78      0.80      1290
weighted avg       0.86      0.86      0.85      1290

